In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
#from textblob import TextBlob

In [2]:
# Load the existing stock data
dataAAPL = pd.read_csv('../data/AAPL_historical_data.csv', index_col='Date', parse_dates=True)
dataAMZN = pd.read_csv('../data/AMZN_historical_data.csv', index_col='Date', parse_dates=True)
dataGOOG = pd.read_csv('../data/GOOG_historical_data.csv', index_col='Date', parse_dates=True)
dataMETA = pd.read_csv('../data/META_historical_data.csv', index_col='Date', parse_dates=True)
dataMSFT = pd.read_csv('../data/MSFT_historical_data.csv', index_col='Date', parse_dates=True)
dataNVDA = pd.read_csv('../data/NVDA_historical_data.csv', index_col='Date', parse_dates=True)
dataTSLA = pd.read_csv('../data/TSLA_historical_data.csv', index_col='Date', parse_dates=True)

In [3]:
# Load the exosting news data
news_data = pd.read_csv('../data/raw_analyst_ratings.csv', parse_dates=['date'])


In [4]:
# Convert to datetime and adjust to UTC
news_data['date'] = pd.to_datetime(news_data['date'], utc=True, format='mixed')
dataAAPL.index = pd.to_datetime(dataAAPL.index, utc=True)


# Extract just the date part
news_data['date'] = news_data['date'].dt.date
# To extract the date index
dates = dataAAPL.index
dataAAPL = dataAAPL.reset_index()

dataAAPL['Date'] = dataAAPL['Date'].dt.date

# Calculate daily percentage changes
dataAAPL['Daily_Return'] = dataAAPL['Close'].pct_change() * 100  # Multiply by 100 to get percentage


# Merge on the date
merged_data = pd.merge(news_data, dataAAPL, left_on='date', right_on='Date', how='left')

# Handle non-trading days by filling missing stock data with the closest trading day
merged_data = merged_data.ffill()  # Forward fill


In [5]:
print("\nMerged Data:")
print(merged_data.head())
print("Date range in merged data:")
print(merged_data['date'].min(), merged_data['date'].max())


Merged Data:
   Unnamed: 0                                           headline  \
0           0            Stocks That Hit 52-Week Highs On Friday   
1           1         Stocks That Hit 52-Week Highs On Wednesday   
2           2                      71 Biggest Movers From Friday   
3           3       46 Stocks Moving In Friday's Mid-Day Session   
4           4  B of A Securities Maintains Neutral on Agilent...   

                                                 url          publisher  \
0  https://www.benzinga.com/news/20/06/16190091/s...  Benzinga Insights   
1  https://www.benzinga.com/news/20/06/16170189/s...  Benzinga Insights   
2  https://www.benzinga.com/news/20/05/16103463/7...         Lisa Levin   
3  https://www.benzinga.com/news/20/05/16095921/4...         Lisa Levin   
4  https://www.benzinga.com/news/20/05/16095304/b...         Vick Meyer   

         date stock        Date       Open       High        Low      Close  \
0  2020-06-05     A  2020-06-05  80.837502  82.

In [6]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [7]:
from nltk.sentiment import SentimentIntensityAnalyzer

# Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()


In [8]:
# Perform sentiment analysis on headlines
def get_sentiment_score(text):
    sentiment = analyzer.polarity_scores(text)
    return sentiment['compound']  # Compound score

# Apply sentiment analysis
merged_data['sentiment_score'] = merged_data['headline'].apply(get_sentiment_score)

# Display the merged DataFrame with sentiment scores
print("Merged Data with Sentiment Scores:")
print(merged_data)

Merged Data with Sentiment Scores:
         Unnamed: 0                                           headline  \
0                 0            Stocks That Hit 52-Week Highs On Friday   
1                 1         Stocks That Hit 52-Week Highs On Wednesday   
2                 2                      71 Biggest Movers From Friday   
3                 3       46 Stocks Moving In Friday's Mid-Day Session   
4                 4  B of A Securities Maintains Neutral on Agilent...   
...             ...                                                ...   
1407323     1413844             Top Narrow Based Indexes For August 29   
1407324     1413845  Recap: Wednesday's Top Percentage Gainers and ...   
1407325     1413846  UPDATE: Oppenheimer Color on China Zenix Auto ...   
1407326     1413847  Oppenheimer Initiates China Zenix At Outperfor...   
1407327     1413848  China Zenix Auto International Opens For Tradi...   

                                                       url          publishe

In [9]:
# Aggregate sentiment scores by date (compute the average)
daily_sentiments = merged_data.groupby('date')['sentiment_score'].mean().reset_index()

# Display the daily sentiment scores
print(daily_sentiments)
print(daily_sentiments['date'].min(), daily_sentiments['date'].max())

            date  sentiment_score
0     2009-02-14         0.226300
1     2009-04-27         0.000000
2     2009-04-29         0.000000
3     2009-05-22         0.000000
4     2009-05-27         0.751050
...          ...              ...
3950  2020-06-07         0.040156
3951  2020-06-08         0.250061
3952  2020-06-09         0.283393
3953  2020-06-10         0.044021
3954  2020-06-11         0.122841

[3955 rows x 2 columns]
2009-02-14 2020-06-11


In [10]:
# Ensure stock data contains 'Date' and 'Daily_Return' columns
dataAAPL = dataAAPL.rename(columns={'Date': 'date'})
dataAAPL = dataAAPL[['date', 'Daily_Return']]  # Ensure only relevant columns are included

In [11]:
# Merge aggregated sentiment scores with stock data
merged_data = pd.merge(daily_sentiments, dataAAPL, on='date', how='left')

In [12]:
# Compute correlation between sentiment scores and daily returns
correlation = merged_data[['sentiment_score', 'Daily_Return']].corr().iloc[0, 1]

In [13]:
# Display results
print("Merged Data with Aggregated Sentiments and Daily Returns:")
print(merged_data)
print(f"\nCorrelation between Sentiment Scores and Daily Returns: {correlation:.2f}")

Merged Data with Aggregated Sentiments and Daily Returns:
            date  sentiment_score  Daily_Return
0     2009-02-14         0.226300           NaN
1     2009-04-27         0.000000      0.669889
2     2009-04-29         0.000000      1.000808
3     2009-05-22         0.000000     -1.352874
4     2009-05-27         0.751050      1.735759
...          ...              ...           ...
3950  2020-06-07         0.040156           NaN
3951  2020-06-08         0.250061      0.591249
3952  2020-06-09         0.283393      3.157800
3953  2020-06-10         0.044021      2.572751
3954  2020-06-11         0.122841     -4.801044

[3955 rows x 3 columns]

Correlation between Sentiment Scores and Daily Returns: 0.15


In [26]:
# Reset the index to make 'Date' a column and rename it to 'date'
daily_returns = dataAAPL.reset_index()[['date', 'Daily_Return']]
daily_returns.head()

,date,Daily_Return
0,1980-12-12,NaN
1,1980-12-15,-5.217061
2,1980-12-16,-7.339788
3,1980-12-17,2.475091
4,1980-12-18,2.899246


In [29]:
#Merge sentiment scores with stock returns
merged_data = pd.merge(daily_sentiments, daily_returns, on='date', how='inner')

# Calculate the Pearson correlation coefficient
correlation = merged_data[['sentiment_score', 'Daily_Return']].corr().iloc[0, 1]
print("Pearson correlation coefficient between average daily sentiment scores and stock daily returns:", correlation)

Pearson correlation coefficient between average daily sentiment scores and stock daily returns: 0.14605816008631772
